# DAY 0 – Setup & Data Loading (Prerequisites)

In [0]:
%pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 93.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "rjxgamerxyt"
os.environ["KAGGLE_KEY"] = "9518dee67c549c7e90b44dee39d99a5e"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

DataFrame[]

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")


DataFrame[]

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors


100%|██████████| 4.29G/4.29G [00:27<00:00, 169MB/s]


In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-1fda727f-509d-4aa4-94e2-48 nogroup 8.4G Jan  9 07:04 2019-Nov.csv
-rwxrwxrwx 1 spark-1fda727f-509d-4aa4-94e2-48 nogroup 5.3G Jan  9 07:07 2019-Oct.csv
-rwxrwxrwx 1 spark-1fda727f-509d-4aa4-94e2-48 nogroup 4.3G Jan  9 07:03 ecommerce-behavior-data-from-multi-category-store.zip


In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

total 14G
-rwxrwxrwx 1 spark-1fda727f-509d-4aa4-94e2-48 nogroup 8.4G Jan  9 07:04 2019-Nov.csv
-rwxrwxrwx 1 spark-1fda727f-509d-4aa4-94e2-48 nogroup 5.3G Jan  9 07:07 2019-Oct.csv


In [0]:
oct_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
    header=True,
    inferSchema=True
)
nov_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
    header=True,
    inferSchema=True
)

In [0]:
display(nov_events.limit(10))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-01T00:00:00.000Z,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2019-11-01T00:00:00.000Z,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2019-11-01T00:00:01.000Z,view,17302664,2053013553853497655,null,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
2019-11-01T00:00:01.000Z,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
2019-11-01T00:00:01.000Z,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
2019-11-01T00:00:01.000Z,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
2019-11-01T00:00:01.000Z,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
2019-11-01T00:00:02.000Z,view,15900065,2053013558190408249,null,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
2019-11-01T00:00:02.000Z,view,12708937,2053013553559896355,null,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24
2019-11-01T00:00:02.000Z,view,1004258,2053013555631882655,electronics.smartphone,apple,732.07,532647354,d2d3d2c6-631d-489e-9fb5-06f340b85be0


In [0]:
print(f"October 2019 - Total Events: {oct_events.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
oct_events.printSchema()

October 2019 - Total Events: 42,448,764

SCHEMA:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
nov_events.show(5, truncate=False)


SAMPLE DATA (First 5 rows):
+-------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code            |brand |price |user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|2019-11-01 00:00:00|view      |1003461   |2053013555631882655|electronics.smartphone   |xiaomi|489.07|520088904|4d3b30da-a5e4-49df-b1a8-ba5943f1dd33|
|2019-11-01 00:00:00|view      |5000088   |2053013566100866035|appliances.sewing_machine|janome|293.65|530496790|8e5f4f83-366c-4f70-860e-ca7417414283|
|2019-11-01 00:00:01|view      |17302664  |2053013553853497655|NULL                     |creed |28.31 |561587266|755422e7-9040-477b-9bd2-6a6e8fd97387|
|2019-11-01 00:00:01|view      |3601530   |2053013563810775923|ap

# Day 1: Platform Setup & First Steps

In [0]:
# Load your data
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)

# Verify it's working
print(f"Ready to go! Loaded {events.count():,} events")
events.show(5)

# Your Day 1 challenges start here...

Ready to go! Loaded 42,448,764 events
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view

In [0]:
del events, oct_events, nov_events

In [0]:
# Create simple DataFrame
data = [("iPhone", 999), ("Samsung", 799), ("MacBook", 1299)]
df = spark.createDataFrame(data, ["product", "price"])
df.show()

# Filter expensive products
df.filter(df.price > 1000).show()

+-------+-----+
|product|price|
+-------+-----+
| iPhone|  999|
|Samsung|  799|
|MacBook| 1299|
+-------+-----+

+-------+-----+
|product|price|
+-------+-----+
|MacBook| 1299|
+-------+-----+

